In [19]:
# Going to be taking the data for yearly stats from basketball reference.com
import requests
from bs4 import BeautifulSoup
years_to_grab = range(2010,2019)

html_format = 'https://www.basketball-reference.com/leagues/NBA_2012_per_game.html'

# iterate through each year
all_soups = {}
for curr_year in years_to_grab:
    # to iterate through each year just replace the year from above
    curr_html = html_format.replace("2012", str(curr_year))
    # get data from proper page  and store in all_soups
    page = requests.get(curr_html)
    all_soups[str(curr_year)] = BeautifulSoup(page.text,'lxml')

In [20]:
# For each year lets extract the data and respective headers
all_data = {}
all_headers = {}
for curr_year in years_to_grab:
    data = []
    headers = []
    table = all_soups[str(curr_year)].find('table')
    table_body = table.find('tbody')
    rows = table_body.find_all('tr')
    nRows = len(rows)
    print nRows

    # to actually extract the data for each year
    for curr_row in rows:
        all_col_that_row = curr_row.find_all('td')
        if len(all_col_that_row) > 0:
            data_curr_row = []
            for curr_col in all_col_that_row:
                # if isnumeric convert to float for later math
                if isfloat(curr_col.text):
                    data_curr_row.append(float(curr_col.text))
                else:
                    data_curr_row.append(curr_col.text)      
            data.append(data_curr_row)
    # now lets save
    all_data[str(curr_year)] = data
    

    # lets also get the header information so can index into each category quickly
    table_head = table.find('thead')
    table_head_row = table_head.find('tr')
    th_col = table_head_row.find_all('th')
    for curr_head in th_col:
        headers.append(curr_head.text)

    # have to remove first value or headers bc rank isn't an actual value in data scraped
    headers.remove('Rk')
    all_headers[str(curr_year)] = headers


600
647
574
596
635
675
601
619
690


In [14]:
# all helper functions for this script
import numpy as np

# quickly test if value is a float
def isfloat(value):
  try:
    float(value)
    return True
  except ValueError:
    return False

# make a function that quickly will give you all values in a column that you care about
# assumes that the length of the headers label matches the number of columns in the dataset
def get_stats_player(dataset,headers,year,stat="all",name="all"):
    if isinstance(year,str):
        curr_data = dataset[year]
        curr_headers = headers[year]
    else:
        curr_data = dataset[str(year)]
        curr_headers = headers[str(year)]
    # if want all stats see what that should be
    if stat == "all":
        stat = [curr_headers[i] for i in range(0,len(curr_data[0])) if isfloat(curr_data[0][i])]
    # check if type of data is a list and if not make it one
    if isinstance(stat, list) == False:
        stat = [stat]
    # these are the columns to use based off the stats selected
    col_to_use = []
    for curr_category in stat:
        col_to_use.append(curr_headers.index(curr_category))
    # now grab that col of the dataset for those players you want
    output_data = []
    for curr_col in col_to_use:
        curr_vec = []
        all_names =[]
        for curr_row in range(0,np.shape(curr_data)[0]):
            curr_vec.append(curr_data[curr_row][curr_col])
            all_names.append(curr_data[curr_row][curr_headers.index("Player")])
        output_data.append(curr_vec)
        if name!= "all":
            final_output = []
            for curr_output in range(0,np.shape(output_data)[0]):
                final_output.append(output_data[curr_output][all_names.index(name)])
        else:
            final_output = output_data
    return final_output


In [22]:
test = get_stats_player(all_data,all_headers,2018,stat="all",name="Draymond Green")
print test
print headers

[27.0, 70.0, 70.0, 32.7, 4.0, 8.8, 0.454, 1.1, 3.7, 0.301, 2.9, 5.2, 0.562, 0.516, 1.9, 2.5, 0.775, 1.1, 6.6, 7.6, 7.3, 1.4, 1.3, 2.9, 2.6, 11.0]
[u'Player', u'Pos', u'Age', u'Tm', u'G', u'GS', u'MP', u'FG', u'FGA', u'FG%', u'3P', u'3PA', u'3P%', u'2P', u'2PA', u'2P%', u'eFG%', u'FT', u'FTA', u'FT%', u'ORB', u'DRB', u'TRB', u'AST', u'STL', u'BLK', u'TOV', u'PF', u'PS/G']
